# Tutorial 6 (Introduction to AI)

# Neural Networks: MLP (Part 2)

## 2. Neural Network classification using Iris

This second example revisits the iris dataset for multi-classification. The key point to note for classification is that since there are three classes, the network needs three outputs, each 0/1, and the output needs to be one-hot encoded.

Two of the helper functions are used here, so they are included.  

In [11]:
import base64
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn import preprocessing

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Convert a Pandas dataframe to the X,y inputs that Keras needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

As noted above, for classification, as compared to regression, the major difference is the output layer.  For regression a single unit computes a linear combination of its input to produce an output value, whereas a classification network needs to decide which of several classes to categorise an input to.  This is achieved by having an output node for each class. These are then considered together with an output **softmax** activation function which results in an output which is the probability of the input belonging to that class.  The loss function for classification network is typicall **categorical crossentropy** (also called **logloss**). 

In the example below using the iris dataset, you can see this illustrated in the context of 5-fold cross-validation.  The number of hidden units, and the number of epochs has here been selected to work well, but you should experiment with changing the values, perhaps also adding another hidden layer.

The species names have been encoded as integers using the encode_text_index method (which is in turn using LabelEncoder).  An alternative using keras's to_categorical is also included.

In [14]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

path = "../ex1/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))

species = encode_text_index(df,"species")
X,y = to_xy(df,"species")

#or directly, using LabelEncoder to transform text to integers, then
#to_categorical to one hot encode
#le = preprocessing.LabelEncoder()
#df['species'] = le.fit_transform(df['species'])
#y = keras.utils.to_categorical(df['species'].to_numpy())
#X = df.drop(columns=['species']).to_numpy()

#Let's look at y
print(y)

print(X.shape)

model = Sequential()
model.add(Input(X[1].shape))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

kf = KFold(5)

for train, test in kf.split(X):
    X_train = X[train]
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]

    model.fit(X_train,y_train,verbose=0,epochs=128)
    pred = model.predict(X_test)
    pred = np.argmax(pred,axis=1)
    y_compare = np.argmax(y_test,axis=1) 
    score = metrics.accuracy_score(y_compare, pred)
    print("Accuracy score: {}".format(score))

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0.

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 515 (2.01 KB)

 Trainable params: 515 (2.01 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Accuracy score: 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Accuracy score: 0.9666666666666667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Accuracy score: 0.9666666666666667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Accuracy score: 0.9333333333333333
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Accuracy score: 1.0


The iris dataset has three classes, so the output from the network consists of three values, one from each output layer node (corresponding to each class).  This number is a probability, so the prediction from this model consist of triples, as below: 

In [15]:
from sklearn import metrics

pred = model.predict(X) # using all the data
print(pred[0:10]) # print first ten predictions

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[8.8084307e-05 9.9486828e-01 5.0436514e-03]
 [9.9982369e-01 1.7631020e-04 6.9708131e-18]
 [2.8508250e-11 5.2625521e-05 9.9994743e-01]
 [1.0457054e-04 9.7744232e-01 2.2453232e-02]
 [4.5501343e-05 9.9602973e-01 3.9247810e-03]
 [9.9952704e-01 4.7294595e-04 1.8631562e-16]
 [1.6055385e-03 9.9831396e-01 8.0457416e-05]
 [6.6318484e-07 3.9269038e-02 9.6073025e-01]
 [4.7802347e-05 7.6981914e-01 2.3013306e-01]
 [3.7060759e-04 9.9946225e-01 1.6699082e-04]]


In my run of this, the first row says: [2.50053126e-04 9.90728140e-01 9.02187731e-03].  That is class 0 is very unlikely to occur with miniscule probability, class 1 has probability of about 99.1%, and class two probability of  less than 1%.  The output prediction here is then class 1 (see below).  Notice that we might potentially ask for a second choice, and that we have data to allow us to make this second choice.

In [16]:
print(np.argmax(pred,axis=1))
print(np.argmax(y,axis=1))

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 1 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 2 2
 1 1 2 1 0 1 2 0 0 1 1 0 2 0 0 2 1 2 2 2 2 1 0 0 2 2 0 0 0 1 2 0 2 2 0 1 1
 2 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2 0 1 2 2 1 2 1 1 2 2 0 1 2 0
 1 2]
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 2 2
 1 1 2 1 0 1 2 0 0 1 1 0 2 0 0 1 1 2 1 2 2 1 0 0 2 2 0 0 0 1 2 0 2 2 0 1 1
 2 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2 0 1 2 2 1 2 1 1 2 2 0 1 2 0
 1 2]


## Load/Save trained network

Complex neural networks will take a long time to fit/train.  It is helpful to be able to save these neural networks so that they can be reloaded later.  A reloaded neural network will not require retraining.  Keras provides at least two formats for neural network saving.

* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Stores the complete neural network (with weights) in the [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). Do not confuse HDF5 with [HDFS](https://en.wikipedia.org/wiki/Apache_Hadoop).  They are different.  We do not use HDFS in this class.

Usually you will want to save in HDF5.

The code below sets up a neural network and reads the data (for predictions), but it does not clear the model directory or fit the neural network.  The weights from the previous fit are used.

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)
 
model = Sequential()
model.add(Input(X[1].shape))
model.add(Dense(64, activation='relu'))
model.add(Dense(10))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train,y_train,verbose=0,epochs=128)
model.summary()
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1) 
score = metrics.accuracy_score(y_compare, pred)
print("Accuracy score: {}".format(score))

#path to where the file will be saved
save_path = "."

# save neural network structure to JSON (no weights)
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"), "w") as json_file:
    json_file.write(model_json)

# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"network.h5"))

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 10)             │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,011 (11.77 KB)

 Trainable params: 1,003 (3.92 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,008 (7.85 KB)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Accuracy score: 1.0


Now we reload the network and perform another prediction.  The Accuracy score should match the previous one exactly if the neural network was really saved and reloaded.

In [23]:
from tensorflow.keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(X_test)
pred = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1) 
score = metrics.accuracy_score(y_compare, pred)
print("Accuracy score: {}".format(score))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Accuracy score: 1.0


## Exercises

1. Use the sklearn digits dataset (sklearn.load_digits()) and build a neural network classifier for this dataset.  You might need to use tensorflow.keras.utils and in particular to_categorical() in order to treat the target labels correctly as a one-hot encoding.